### **Install TensorFlow**

In [22]:
pip install tensorflow

### **Install Librabries**

In [3]:
import io
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from sklearn import preprocessing
from google.colab import files

### **Upload Dataset**

In [4]:
upl = files.upload()

Saving pokemon_alopez247.csv to pokemon_alopez247.csv


In [5]:
df = pd.read_csv(io.BytesIO(upl['pokemon_alopez247.csv']))
print(df)

     Number        Name   Type_1  ... Weight_kg  Catch_Rate       Body_Style
0         1   Bulbasaur    Grass  ...       6.9          45        quadruped
1         2     Ivysaur    Grass  ...      13.0          45        quadruped
2         3    Venusaur    Grass  ...     100.0          45        quadruped
3         4  Charmander     Fire  ...       8.5          45   bipedal_tailed
4         5  Charmeleon     Fire  ...      19.0          45   bipedal_tailed
..      ...         ...      ...  ...       ...         ...              ...
716     717     Yveltal     Dark  ...     203.0          45        two_wings
717     718     Zygarde   Dragon  ...     305.0           3  serpentine_body
718     719     Diancie     Rock  ...       8.8           3        head_arms
719     720       Hoopa  Psychic  ...       9.0           3        head_only
720     721   Volcanion     Fire  ...     195.0           3        quadruped

[721 rows x 23 columns]


In [6]:
df.columns

Index(['Number', 'Name', 'Type_1', 'Type_2', 'Total', 'HP', 'Attack',
       'Defense', 'Sp_Atk', 'Sp_Def', 'Speed', 'Generation', 'isLegendary',
       'Color', 'hasGender', 'Pr_Male', 'Egg_Group_1', 'Egg_Group_2',
       'hasMegaEvolution', 'Height_m', 'Weight_kg', 'Catch_Rate',
       'Body_Style'],
      dtype='object')

In [7]:
df = df[['isLegendary','Generation', 'Type_1', 'Type_2', 'HP', 'Attack', 'Defense', 'Sp_Atk', 'Sp_Def', 'Speed','Color','Egg_Group_1','Height_m','Weight_kg','Body_Style']]


### **Checking for Legendary player & creating a dummy Dataset **

In [8]:
df['isLegendary'] = df['isLegendary'].astype(int)


In [9]:
def dummy_creation(df, dummy_categories):
    for i in dummy_categories:
        df_dummy = pd.get_dummies(df[i])
        df = pd.concat([df,df_dummy],axis=1)
        df = df.drop(i, axis=1)
    return(df)


In [10]:
df = dummy_creation(df, ['Egg_Group_1', 'Body_Style', 'Color','Type_1', 'Type_2'])


# Split and Normalize Data

In [11]:
def train_test_splitter(DataFrame, column):
    df_train = DataFrame.loc[df[column] != 1]
    df_test = DataFrame.loc[df[column] == 1]

    df_train = df_train.drop(column, axis=1)
    df_test = df_test.drop(column, axis=1)

    return(df_train, df_test)

df_train, df_test = train_test_splitter(df, 'Generation')

In [12]:
def label_delineator(df_train, df_test, label):
    
    train_data = df_train.drop(label, axis=1).values
    train_labels = df_train[label].values
    test_data = df_test.drop(label,axis=1).values
    test_labels = df_test[label].values
    return(train_data, train_labels, test_data, test_labels)

In [13]:
train_data, train_labels, test_data, test_labels = label_delineator(df_train, df_test, 'isLegendary')
print(test_data)

[[ 45.  49.  49. ...   0.   0.   0.]
 [ 60.  62.  63. ...   0.   0.   0.]
 [ 80.  82.  83. ...   0.   0.   0.]
 ...
 [ 91. 134.  95. ...   0.   0.   0.]
 [106. 110.  90. ...   0.   0.   0.]
 [100. 100. 100. ...   0.   0.   0.]]


In [14]:
def data_normalizer(train_data, test_data):
    train_data = preprocessing.MinMaxScaler().fit_transform(train_data)
    test_data = preprocessing.MinMaxScaler().fit_transform(test_data)
    return(train_data, test_data)

train_data, test_data = data_normalizer(train_data, test_data)


In [15]:
length = train_data.shape[1]

model = keras.Sequential()
model.add(keras.layers.Dense(500, activation='relu', input_shape=[length,]))
model.add(keras.layers.Dense(2, activation='softmax'))


In [16]:
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [17]:
model.fit(train_data, train_labels, epochs=400)

Epoch 1/400
18/18 [==============================] - 1s 1ms/step - loss: 0.6125 - accuracy: 0.7895
Epoch 2/400
18/18 [==============================] - 0s 1ms/step - loss: 0.4669 - accuracy: 0.9263
Epoch 3/400
18/18 [==============================] - 0s 2ms/step - loss: 0.3908 - accuracy: 0.9263
Epoch 4/400
18/18 [==============================] - 0s 1ms/step - loss: 0.3474 - accuracy: 0.9263
Epoch 5/400
18/18 [==============================] - 0s 1ms/step - loss: 0.3209 - accuracy: 0.9263
Epoch 6/400
18/18 [==============================] - 0s 1ms/step - loss: 0.3033 - accuracy: 0.9263
Epoch 7/400
18/18 [==============================] - 0s 2ms/step - loss: 0.2911 - accuracy: 0.9263
Epoch 8/400
18/18 [==============================] - 0s 2ms/step - loss: 0.2816 - accuracy: 0.9263
Epoch 9/400
18/18 [==============================] - 0s 2ms/step - loss: 0.2741 - accuracy: 0.9263
Epoch 10/400
18/18 [==============================] - 0s 2ms/step - loss: 0.2679 - accuracy: 0.9263
Epoch 11/

In [19]:
loss_value, accuracy_value = model.evaluate(test_data, test_labels)
print(f'Our test accuracy was{accuracy_value}')

5/5 [==============================] - 0s 2ms/step - loss: 0.0367 - accuracy: 0.9801
Our test accuracy was0.9801324605941772


In [20]:
def predictor(test_data, test_labels, index):
    prediction = model.predict(test_data)
    if np.argmax(prediction[index]) == test_labels[index]:
        print(f'This was correctly predicted to be a \"{test_labels[index]}\"!')
    else:
        print(f'This was incorrectly predicted to be a \"{np.argmax(prediction[index])}\". It was actually a \"{test_labels[index]}\".')
        return(prediction)


In [21]:
predictor(test_data, test_labels, 149)

This was correctly predicted to be a "1"!
